# Clustering collums and rows

The idea is to cluster collumns and rows

Here we have the objective function 



In [ ]:
import numpy as np
import tvsclib.utils as utils
import Split
import matplotlib.pyplot as plt
from tvsclib.strict_system import StrictSystem

from tvsclib.approximation import Approximation

import torchvision.models as models
import torch
import scipy.stats 

import graphs

import scipy.linalg as linalg

import plot_permutations as perm

In [ ]:
def get_mobilenet_target_mats():
    target_mats = []
    # Load the model
    model = models.mobilenet_v2(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats

In [ ]:
T = np.random.rand(32,32)
#T = mats = get_mobilenet_target_mats()[0]
sys = Split.initial_mixed(T)
utils.show_system(sys,mark_D=False)
utils.check_dims(sys)

# Compute gramians/weights

In [ ]:
#get matricies
k = 0
stage_c=sys.causal_system.stages[k]
stage_a=sys.anticausal_system.stages[k]
Ac = stage_c.A_matrix
Bc = stage_c.B_matrix
Cc = stage_c.C_matrix

Aa = stage_a.A_matrix
Ba = stage_a.B_matrix
Ca = stage_a.C_matrix

D = stage_c.D_matrix

#dims of states

(d_out_c,d_in_c) = Ac.shape

(d_out_a,d_in_a) = Aa.shape

X = np.block([[np.zeros((d_out_a,d_in_c)),Ba,Aa ],
              [Cc,D,Ca],
              [Ac,Bc,np.zeros((d_out_c,d_in_a))]
    
])

plt.matshow(X)

s_c = np.zeros(X.shape[1],dtype=bool)
s_c[:d_in_c+D.shape[1]//2]=1
s_r = np.zeros(X.shape[0],dtype=bool)
s_r[d_out_a+D.shape[0]//2:]=1

In [ ]:
X_l = X[:,s_c]
X_r = X[:,~s_c]

X_l = X_l/np.linalg.norm(X_l,axis=1).reshape(-1,1)
X_r = X_r/np.linalg.norm(X_r,axis=1).reshape(-1,1)

X_t = X[~s_r]
X_b = X[s_r]

X_t = X_t/np.linalg.norm(X_t,axis=0).reshape(1,-1)
X_b = X_b/np.linalg.norm(X_b,axis=0).reshape(1,-1)

In [ ]:
X_ = X_b #top or bottom
print(np.diag(X_.T@X_))
plt.matshow(X_.T@X_)

In [ ]:
X_ = X_r  #left or right
print(np.diag(abs(X_@X_.T)))
plt.matshow(X_@X_.T)

In [ ]:
plt.matshow(1-abs(X_@X_.T))

In [ ]:
Adj = 1-abs(X_b.T@X_b)
np.sum(Adj[:,s_c],axis=1)

In [ ]:
Adj = 1-abs(X_t.T@X_t)
np.sum(Adj[:,~s_c],axis=1)

In [ ]:
Adj = 1-abs(X_l@X_l.T)
np.sum(Adj[:,s_r],axis=1)

In [ ]:
Adj = 1-abs(X_r@X_r.T)
np.sum(Adj[:,~s_r],axis=1)

## Some helping functions

In [ ]:
def f(X,s_col,s_row):
    return 0

def compute_sigmasf(X,s_col,s_row):
    return np.linalg.svd(X[s_row][:,s_col],compute_uv=False), np.linalg.svd(X[~s_row][:,~s_col],compute_uv=False)

def show_matrices(X,s_col,s_row):
    plt.matshow(X[s_row][:,s_col],fignum=1)

    plt.matshow(X[~s_row][:,~s_col],fignum=2)

f_reg_row = lambda x: -gamma*x**3
f_reg_col = lambda x: -gamma*x**3

gamma = 1e4 #maxbee have two different regularizations for rows and collumns

## Iteration

In [ ]:
def segment_matrix(stage_causal,stage_anticausal,N=70,initla_spectral=True):
    
    Ac = stage_causal.A_matrix
    Bc = stage_causal.B_matrix
    Cc = stage_causal.C_matrix

    Aa = stage_anticausal.A_matrix
    Ba = stage_anticausal.B_matrix
    Ca = stage_anticausal.C_matrix

    D = stage_causal.D_matrix
    
    #regularization vector
    v_reg_col = f_reg_col(np.linspace(-1,1,D.shape[1]))
    v_reg_row = f_reg_row(np.linspace(-1,1,D.shape[0]))

    #dims of states
    (d_out_c,d_in_c) = Ac.shape
    (d_out_a,d_in_a) = Aa.shape

    #setup matrix
    X = np.block([[np.zeros((d_out_a,d_in_c)),Ba,Aa ],
                  [Cc,D,Ca],
                  [Ac,Bc,np.zeros((d_out_c,d_in_a))]
    
    ])

    if initla_spectral:
        #get initial using spectral 
        s_c = graphs.segment_matrix(X)
        if np.count_nonzero(~s_c[:d_in_c])+np.count_nonzero(s_c[X.shape[1]-d_in_a:]) > (d_in_a+d_in_c)/2:
            # if more fixed nodes are incorrect flip
            s_c = ~s_c
            
        #set the fixed
        s_c[:d_in_c]=1
        s_c[X.shape[1]-d_in_a:]=0
        
        s_r = graphs.segment_matrix(X.T)
        if np.count_nonzero(s_r[:d_out_a])+np.count_nonzero(~s_r[X.shape[0]-d_out_c:]) > (d_out_a+d_out_c)/2:
            # if more fixed nodes are incorrect flip
            s_r = ~s_r
        s_r[:d_out_a]=0
        s_r[X.shape[0]-d_out_c:]=1
    else:
        #initialize segmentation
        s_c = np.zeros(X.shape[1],dtype=bool)
        s_c[:d_in_c+D.shape[1]//2]=1
        s_r = np.zeros(X.shape[0],dtype=bool)
        s_r[d_out_a+D.shape[0]//2:]=1
    
    
    fs = np.zeros(N+1)
    s_cols=np.zeros((N+1,X.shape[1]),dtype=bool) 
    s_rows=np.zeros((N+1,X.shape[0]),dtype=bool) 
    

    s_cols[0]=s_c
    s_rows[0]=s_r
    fs[0]=f(X,s_c,s_r)
    
    for n in range(N):
        #for test 
        normalize ="F"
        
        #columns:
        X_t = X[~s_r]
        X_b = X[s_r]
        
        n_xt =np.linalg.norm(X_t,axis=0)
        n_xb =np.linalg.norm(X_b,axis=0)
        
        #weights unnormalized
        W_t = n_xt.reshape(-1,1)*n_xt.reshape(1,-1)-np.abs(X_t.T@X_t)
        W_b = n_xb.reshape(-1,1)*n_xb.reshape(1,-1)-np.abs(X_b.T@X_b)
        
        #nomalize
        if normalize =="F": #Forbenius norm
            W_t = W_t/(n_xt.reshape(-1,1)*n_xt.reshape(1,-1))
            W_b = W_b/(n_xb.reshape(-1,1)*n_xb.reshape(1,-1))
        elif normalize =="L": #length
            W_t = W_t/(np.count_nonzero(~s_r)**2)
            W_b = W_b/(np.count_nonzero(s_r)**2)
        elif normalize =="M": #mixed: reference with norm, current with length
            W_t = W_t/(n_xt.reshape(-1,1)*np.count_nonzero(~s_r))#
            W_b = W_b/(n_xb.reshape(-1,1)*np.count_nonzero(s_r))
            
        #rows:
        X_r = X[:,~s_c]
        X_l = X[:,s_c]
        
        n_xr = np.linalg.norm(X_r,axis=1)
        n_xl = np.linalg.norm(X_l,axis=1)
        
        #Weights unnormalized
        W_r =n_xr.reshape(-1,1)*n_xr.reshape(1,-1)-np.abs(X_r@X_r.T)
        W_l =n_xl.reshape(-1,1)*n_xl.reshape(1,-1)-np.abs(X_l@X_l.T)
        
        #normlaize
        if normalize =="F": #Forbenius norm
            W_r = W_r/(n_xr.reshape(-1,1)*n_xr.reshape(1,-1))
            W_l = W_l/(n_xl.reshape(-1,1)*n_xl.reshape(1,-1))
        elif normalize =="L": #length
            W_r = W_r/(np.count_nonzero(~s_c)**2)#
            W_l = W_l/(np.count_nonzero(s_c)**2)
        elif normalize =="M": #mixed: reference with norm, current with length
            W_r = W_r/(n_xr.reshape(-1,1)*np.count_nonzero(~s_c))#
            W_l = W_l/(n_xl.reshape(-1,1)*np.count_nonzero(s_c))

        S_col = np.sum(W_t[~s_c],axis=0) -np.sum(W_b[s_c],axis=0)
        S_row = np.sum(W_r[~s_r],axis=0) -np.sum(W_l[s_r],axis=0)
        
        ord_c = d_in_c +np.argsort(S_col[d_in_c:X.shape[1]-d_in_a])
        ord_r = d_out_a+np.argsort(S_row[d_out_a:X.shape[0]-d_out_c])
        
        
        v_c = ord_c[v_reg_col<S_col[ord_c]]
        v_r = ord_r[v_reg_row<S_row[ord_r]]
        
        s_c[d_in_c:s_c.size-d_in_a]=0
        s_r[d_out_a:s_r.size-d_out_c]=0
        
        s_c[v_c] = 1
        s_r[v_r] = 1
        
        
        fs[n+1] = f(X,s_c,s_r)
        s_cols[n+1]=s_c
        s_rows[n+1]=s_r
        
        if np.any(np.logical_and(np.all(s_cols[:n+1]==s_c,axis=1),np.all(s_rows[:n+1]==s_r,axis=1))):
            print("converged at n=",n)
            break
        
    report ={"s_cols":s_cols[:n+2],"s_rows":s_rows[:n+2],"X":X}
        
    return s_c[d_in_c:X.shape[1]-d_in_a],s_r[d_out_a:X.shape[0]-d_out_c],report
        


In [ ]:
sys = Split.initial_mixed(T)
stage_c=sys.causal_system.stages[k]
stage_a=sys.anticausal_system.stages[k]
s_c,s_r,report = segment_matrix(stage_c,stage_a,N=20,initla_spectral=True)
s_cols = report["s_cols"]
s_rows = report["s_rows"]

In [ ]:
s_start_c,s_start_a = compute_sigmasf(X,s_cols[0],s_rows[0])
s_end_c,s_end_a = compute_sigmasf(X,s_cols[-1],s_rows[-1])

plt.subplot(1,2,1)
plt.plot(s_start_c,label="start")
plt.plot(s_end_c,label="end")
plt.legend()
plt.subplot(1,2,2)
plt.plot(s_start_a,label="start")
plt.plot(s_end_a,label="end")

In [ ]:
plt.spy(s_cols)
print(np.count_nonzero(s_cols[-1])/len(s_cols[-1]))

In [ ]:
plt.spy(s_rows)
print(np.count_nonzero(s_cols[-1])/len(s_cols[-1]))

## Some helping function to apply permutation and store it

In [ ]:
def get_permutations(s_c,s_r):
    p_col = np.hstack([np.nonzero(s_c),np.nonzero(~s_c)]).reshape(-1)
    p_row = np.hstack([np.nonzero(~s_r),np.nonzero(s_r)]).reshape(-1)
    i_in =np.count_nonzero(s_c)
    i_out=np.count_nonzero(s_r)
    return p_col,p_row,i_in,i_out

def permute_stage(stage,p_col,p_row):
    stage.B_tilde = stage.B_tilde[:,p_col]
    stage.C_tilde = stage.C_tilde[p_row,:]
    stage.D_matrix = stage.D_matrix[:,p_col][p_row]
    
def collect_permutations(P_col,P_row,k,p_col,p_row,system):
    """
    Function to collect the permutations in P_col and P_row
    
    P_col:    total permutation of columns
    P_row:    total permutation of columns
    l:        index of stage
    p_col:    new collumn permuation
    p_row:    new row permutation
    """
    
    dims_in = system.dims_in
    dims_out = system.dims_out
    
    I = np.sum(dims_in[:k]).astype(int)
    P_col[I:I+dims_in[k]]=P_col[I:I+dims_in[k]][p_col]
    I = np.sum(dims_out[:k]).astype(int)
    P_row[I:I+dims_out[k]]=P_row[I:I+dims_out[k]][p_row]

In [ ]:
N_col=5
N_row=4
s_c = np.array([0,1]*N_col,dtype=bool)
s_r = np.array([0,1]*N_row,dtype=bool)
A = np.zeros((1,1))
B = np.zeros((1,2*N_col))
C = np.zeros((2*N_row,1))
D = np.zeros((2*N_row,2*N_col))

B[:,s_c]=np.arange(1,N_col+1)
B[:,~s_c]=np.arange(N_col+1,2*N_col+1)

C[s_r]=np.arange(1,N_row+1).reshape(-1,1)
C[s_r]=np.arange(N_row+1,2*N_row+1).reshape(-1,1)

index = np.arange(4*N_col*N_row,dtype=int).reshape(2*N_row,2*N_col)
D.reshape(-1)[index[s_r][:,s_c].reshape(-1)]=(np.arange(1,N_col+1).reshape(1,-1)*np.arange(1,N_row+1).reshape(-1,1)).reshape(-1)
D.reshape(-1)[index[~s_r][:,~s_c].reshape(-1)]=(np.arange(N_col+1,2*N_col+1).reshape(1,-1)*np.arange(N_row+1,2*N_row+1).reshape(-1,1)).reshape(-1)
#D[s_r][:,s_c]#D[~s_r][:,~s_c]

stage = Split.Stage_sigmas(A,B,C,D,np.ones(1),np.ones(1))
p_col,p_row,i_in,i_out = get_permutations(s_c,s_r)
permute_stage(stage,p_col,p_row)
display(stage.A_matrix)
display(stage.B_matrix)
display(stage.C_matrix)
display(stage.D_matrix)


system = StrictSystem(stages=[
    Split.Stage_sigmas(np.zeros((1,0)),np.zeros((1,1)),np.zeros((2,0)),np.zeros((2,1)),np.zeros(1),np.zeros(0)),
    stage,
    Split.Stage_sigmas(np.zeros((0,1)),np.zeros((0,1)),np.zeros((2,1)),np.zeros((2,1)),np.zeros(1),np.zeros(0))
],causal=True)
utils.show_system(system,mark_D=False)

T=linalg.block_diag(np.zeros((2,1)),D,np.zeros((2,1)))
P_col = np.arange(T.shape[1],dtype=int)
P_row = np.arange(T.shape[0],dtype=int)
collect_permutations(P_col,P_row,1,p_col,p_row,system)
display(P_col)
display(P_row)

In [ ]:
plt.matshow(T[P_row][:,P_col])

## Get test matrix

In [ ]:
dims_in =  np.array([4, 4, 4, 4])*6
dims_out = np.array([4, 4, 4, 4])*6

#dims_in =  np.array([9, 7, 7, 9])*3
#dims_out = np.array([7, 9, 9, 7])*3


n = 2
#create orthogonal vectors and normalize them to the size of the matix (i.e. norm(block)/size(block) = const
Us =np.vstack([scipy.stats.ortho_group.rvs(dims_out[i])[:,:3*n]*dims_out[i] for i in range(len(dims_in))])
Vts=np.hstack([scipy.stats.ortho_group.rvs(dims_in[i])[:3*n,:]*dims_in[i] for i in range(len(dims_in))])

s = np.linspace(1,0.75,n)

lower = Us[:,:n]@np.diag(s)@Vts[:n,:]
diag = Us[:,n:2*n]@np.diag(s)@Vts[n:2*n,:]
upper = Us[:,2*n:3*n]@np.diag(s)@Vts[2*n:3*n,:]
matrix = np.zeros_like(diag)
a=0;b=0
for i in range(len(dims_in)):
    matrix[a:a+dims_out[i],:b]            =lower[a:a+dims_out[i],:b]
    matrix[a:a+dims_out[i],b:b+dims_in[i]]=diag[a:a+dims_out[i],b:b+dims_in[i]]
    matrix[a:a+dims_out[i],b+dims_in[i]:] =upper[a:a+dims_out[i],b+dims_in[i]:]
    a+=dims_out[i];b+=dims_in[i]
plt.figure()

P_in_ref = np.random.permutation(np.arange(matrix.shape[1]))
P_out_ref= np.random.permutation(np.arange(matrix.shape[0]))

T = matrix[P_out_ref][:,P_in_ref]
plt.matshow(T)
print(T.shape)

In [ ]:
#T = np.random.rand(32,32)
#T =np.arange(1,32).reshape(1,-1)*np.arange(1,32).reshape(-1,1)
#T = matrix.T
#T = mats = get_mobilenet_target_mats()[0]
sys = Split.initial_sigmas_mixed(T)
utils.show_system(sys,mark_D=False)
utils.check_dims(sys)

## combine it to algorithm

In [ ]:
def identification_split_system(sys,N):
    
    P_col = np.arange(np.sum(sys.dims_in) ,dtype=int)
    P_row = np.arange(np.sum(sys.dims_out),dtype=int)
    
    Ps_col =np.zeros((N,P_col.size),dtype=int)
    Ps_row =np.zeros((N,P_row.size),dtype=int)
    for n in range(N):
        print(n)
        for k in range(len(sys.causal_system.stages)-1,-1,-1): #reverse ordering makes indexing easier 
            i_in =sys.causal_system.stages[k].dim_in//2
            i_out=sys.causal_system.stages[k].dim_out//2
            Split.split_sigmas_mixed(sys,k,i_in,i_out)
    

In [ ]:
sys = Split.initial_sigmas_mixed(T)
identification_split_system(sys,2)
utils.check_dims(sys)
utils.show_system(sys)

In [ ]:
sigmas_causal =[stage.s_in for stage in sys.causal_system.stages][1:]
sigmas_anticausal =[stage.s_in for stage in sys.anticausal_system.stages][:-1]
#print(sigmas_causal)
#print(sigmas_anticausal)
plt.subplot(1,2,1)
for sig in sigmas_causal:
    plt.scatter(np.arange(len(sig)),sig)
plt.subplot(1,2,2)
for sig in sigmas_anticausal:
    plt.scatter(np.arange(len(sig)),sig)

In [ ]:
def identification_split_clustering(sys,N):
    
    P_col = np.arange(np.sum(sys.dims_in) ,dtype=int)
    P_row = np.arange(np.sum(sys.dims_out),dtype=int)
    
    Ps_col =np.zeros((N+1,P_col.size),dtype=int)
    Ps_row =np.zeros((N+1,P_row.size),dtype=int)
    Ps_col[0]=P_col
    Ps_row[0]=P_row
    reports = []
    for n in range(N):
        print(n)
        for k in range(len(sys.causal_system.stages)-1,-1,-1): #reverse ordering makes indexing easier 
            stage_c=sys.causal_system.stages[k]
            stage_a=sys.anticausal_system.stages[k]
            s_c,s_r,report = segment_matrix(stage_c,stage_a,N=20)
            reports.append(report)
            
            assert len(s_c)==stage_c.dim_in ,"dims_in causal do not match s_c"
            assert len(s_r)==stage_c.dim_out,"dims_out causal do not match s_r"
            assert len(s_c)==stage_a.dim_in ,"dims_in antic do not match s_c"
            assert len(s_r)==stage_a.dim_out,"dims_out antic do not match s_r"
            p_col,p_row,i_in,i_out = get_permutations(s_c,s_r)
            permute_stage(stage_c,p_col,p_row)
            permute_stage(stage_a,p_col,p_row)
            collect_permutations(P_col,P_row,k,p_col,p_row,sys)
            
            Split.split_sigmas_mixed(sys,k,i_in,i_out)
        #save the Permutations collected for all stages
        Ps_col[n+1]=P_col
        Ps_row[n+1]=P_row
    
    return Ps_col,Ps_row,reports

In [ ]:
gamma = 1e6
sys = Split.initial_sigmas_mixed(T)
Ps_col,Ps_row,reports = identification_split_clustering(sys,2)
P_col = Ps_col[-1]
P_row = Ps_row[-1]
utils.check_dims(sys)
utils.show_system(sys)

In [ ]:
plt.subplot(1,2,1)
plt.scatter(np.arange(len(P_col)),P_col)
plt.subplot(1,2,2)
plt.scatter(np.arange(len(P_row)),P_row)

In [ ]:
sigmas_causal =[stage.s_in for stage in sys.causal_system.stages][1:]
sigmas_anticausal =[stage.s_in for stage in sys.anticausal_system.stages][:-1]
#print(sigmas_causal)
#print(sigmas_anticausal)
plt.subplot(1,2,1)
for i,sig in enumerate(sigmas_causal):
    plt.scatter(np.arange(len(sig)),sig,label=str(i))
plt.legend()
plt.subplot(1,2,2)
for i,sig in enumerate(sigmas_anticausal):
    plt.scatter(np.arange(len(sig)),sig,label=str(i))
plt.legend()

In [ ]:
plt.matshow(T[P_row][:,P_col])

In [ ]:
plt.matshow(T[Ps_row[-1]][:,Ps_col[-1]])

In [ ]:
utils.check_dims(sys)

In [ ]:
cmap = plt.cm.get_cmap('tab20')
colors = np.repeat(cmap((1/20)*np.arange(4)+0.001),dims_in,axis=0)[P_in_ref]
perm.multiple_connection_plot(perm.invert_permutations(Ps_col),colors=colors,start=0,end=2)

In [ ]:
cmap = plt.cm.get_cmap('tab20')
colors = np.repeat(cmap((1/20)*np.arange(4)+0.001),dims_out,axis=0)[P_out_ref]
perm.multiple_connection_plot(perm.invert_permutations(Ps_row),colors=colors,start=0,end=2)

# Weight matrix form Mobilenet

In [ ]:
T = get_mobilenet_target_mats()[0]

In [ ]:
sys = Split.initial_sigmas_mixed(T)
identification_split_system(sys,3)
utils.check_dims(sys)
utils.show_system(sys)

In [ ]:
sigmas_causal =[stage.s_in for stage in sys.causal_system.stages][1:]
sigmas_anticausal =[stage.s_in for stage in sys.anticausal_system.stages][:-1]
#print(sigmas_causal)
#print(sigmas_anticausal)
plt.subplot(1,2,1)
for sig in sigmas_causal:
    plt.scatter(np.arange(len(sig)),sig)
plt.subplot(1,2,2)
for sig in sigmas_anticausal:
    plt.scatter(np.arange(len(sig)),sig)

In [ ]:
gamma = 5e3

sys_per = Split.initial_sigmas_mixed(T)
Ps_col,Ps_row,reporst = identification_split_clustering(sys_per,3)
utils.check_dims(sys_per)
utils.show_system(sys_per)

In [ ]:
sigmas_causal =[stage.s_in for stage in sys_per.causal_system.stages][1:]
sigmas_anticausal =[stage.s_in for stage in sys_per.anticausal_system.stages][:-1]
#print(sigmas_causal)
#print(sigmas_anticausal)
plt.subplot(1,2,1)
for sig in sigmas_causal:
    plt.scatter(np.arange(len(sig)),sig)
plt.subplot(1,2,2)
for sig in sigmas_anticausal:
    plt.scatter(np.arange(len(sig)),sig)

In [ ]:
sigmas_causal_per =[stage.s_in for stage in sys_per.causal_system.stages][1:]
sigmas_anticausal_per =[stage.s_in for stage in sys_per.anticausal_system.stages][:-1]

sigmas_causal =[stage.s_in for stage in sys.causal_system.stages][1:]
sigmas_anticausal =[stage.s_in for stage in sys.anticausal_system.stages][:-1]

plt.figure(figsize=[12,8])

plt.subplot(1,2,1)
plt.grid()
for sig in sigmas_causal:
    plt.plot(np.arange(len(sig)),sig,color='C0')
for sig in sigmas_causal_per:
    plt.plot(np.arange(len(sig)),sig,color='C1')
plt.subplot(1,2,2)
for sig in sigmas_anticausal:
    plt.plot(np.arange(len(sig)),sig,color='C0')
for sig in sigmas_anticausal_per:
    plt.plot(np.arange(len(sig)),sig,color='C1')
plt.grid()


In [ ]:
T_per = T[Ps_row[-1]][:,Ps_col[-1]]
np.max(np.abs(T_per-sys_per.to_matrix()))

In [ ]:
eps_max = max([np.max(sig)for sig in sigmas_causal]+[np.max(sig)for sig in sigmas_anticausal])

In [ ]:
approx =Approximation(sys,(sigmas_causal,sigmas_anticausal))
approx_per=Approximation(sys_per,(sigmas_causal_per,sigmas_anticausal_per))


N = 9 #number of points
alpha = np.linspace(0,1,N)

err_move =np.zeros_like(alpha)

eps = eps_max*alpha

def calc_values(approx,eps,matrix):
    costs =np.zeros_like(eps)
    err =np.zeros_like(eps)
    for i in range(len(eps)):
        approx_system=approx.get_approxiamtion(eps[i])
        matrix_approx = approx_system.to_matrix()
        err[i] = np.linalg.norm(matrix_approx-matrix,ord=2)
        costs[i] = approx_system.cost()
    return err,costs

err_orig,cost_orig = calc_values(approx,eps,T)
err_per,cost_per = calc_values(approx_per,eps,T_per)

In [ ]:
plt.plot(cost_orig,err_orig,label="orig")
plt.plot(cost_per,err_per,label="per")
plt.legend()
plt.grid()

In [ ]:
alpha[2]

In [ ]:
err_orig[2]

In [ ]:
err_per[2]

In [ ]:
cost_orig[2]

In [ ]:
cost_per[2]

In [ ]:
i = 2
print(alpha[i])
print(1-cost_per[i]/cost_orig[i])

In [ ]:
print(err_per[i]/err_orig[i])

In [ ]:
plt.plot(alpha,cost_per/cost_orig)